# Cart Pole

#### Descripcion 


+ **Espacio de observacion (Box(4,))**

+ Posicion del carro: [-2.4, 2.4]
+ Velocidad del carro: [$-\infty$, $\infty$]
+ Angulo del palo: [-41.8, 41.8]
+ Velocidad del palo en la punta: [$-\infty$, $\infty$]


+ **Espacio de accion (Discrete(2))**

+ Izquierda: 0
+ Derecha: 1


El objetivo es mantener el palo vertical moviendo a izquierda y derecha el carro.


La recompensa es +1 para cada paso temporal. El episodio se termina si el angulo es mayor a $\pm 12$ grados o si el carro sobrepasa la posicion $\pm 2.4$

In [1]:
# primero test aleatorio del entorno

import gym

In [2]:
entorno=gym.make('CartPole-v0')

In [3]:
for episodio in range(5):  # 5 episodios
    
    done=False
    
    observacion=entorno.reset()  # observacion
    
    recompensa_total=0.          # recompensa total en cada episodio
    
    paso=0                       # paso en cada episodio
     
    while not done:
        entorno.render()  # muestra el entorno
        
        accion=entorno.action_space.sample() # accion aleatoria, se cambiara por el agente
        
        siguiente_estado, recompensa, done, info=entorno.step(accion)
        
        recompensa_total+=recompensa
        
        paso+=1
        
        observacion=siguiente_estado
        
    print ('\nEpisodio #{} finalizado en {} pasos. Recompensa total:{}'.format(episodio+1, paso, recompensa_total))
    
entorno.close()


Episodio #1 finalizado en 29 pasos. Recompensa total:29.0

Episodio #2 finalizado en 21 pasos. Recompensa total:21.0

Episodio #3 finalizado en 13 pasos. Recompensa total:13.0

Episodio #4 finalizado en 20 pasos. Recompensa total:20.0

Episodio #5 finalizado en 18 pasos. Recompensa total:18.0


**Se usara un perceptron desde SkLearn, de una manera mas simple**

In [4]:
import numpy as np
from sklearn.neural_network import MLPClassifier

In [5]:
dim_accion=entorno.action_space.n


#Se crea el agente
agente=MLPClassifier(hidden_layer_sizes=(40,40), activation='relu',
                     solver='adam', warm_start=True, max_iter=50)


#Se entrena el agente de manera aleatoria
agente.fit([entorno.reset()]*dim_accion,range(dim_accion));

In [6]:
def sesion():

    observaciones=[]
    acciones=[]
    recompensa_total=0.
    
    observacion=entorno.reset()   
    
    while 1:
        accion=np.random.choice(dim_accion, p=agente.predict_proba([observacion])[0])
        
        observaciones.append(observacion)
        acciones.append(accion)
        
        siguiente_obs, recompensa, done, info=entorno.step(accion)
        recompensa_total+=recompensa
        
        if done:
            break
        
    return observaciones, acciones, recompensa_total

In [7]:
# Loop de entrenamiento

pasos=100       # se toman 100 pasos
percentil=70    # se entrena en el mejor 30% (30 mejores pasos)


for i in range(50):  # 50 episodios

    sesiones=[sesion() for _ in range(pasos)]
    
    batch_observaciones, batch_acciones, batch_recompensas=map(np.array,zip(*sesiones))
    
    # Se escoge el umbral de recompensas
    umbral=np.percentile(batch_recompensas, percentil)
    
    mejores_observaciones=np.concatenate(batch_observaciones[batch_recompensas>=umbral])
    mejores_acciones=np.concatenate(batch_acciones[batch_recompensas>=umbral])
    
    # Se entrena con lo mejor
    agente.fit(mejores_observaciones, mejores_acciones)

    print('Episodio: {} \tRecompensa Media={}\tRecompensa Max={}\tUmbral={}'.format(i, batch_recompensas.mean(), batch_recompensas.max(), umbral))
    
entorno.close()

Episodio: 0 	Recompensa Media=19.87	Recompensa Max=45.0	Umbral=23.0
Episodio: 1 	Recompensa Media=21.22	Recompensa Max=56.0	Umbral=24.299999999999997
Episodio: 2 	Recompensa Media=22.21	Recompensa Max=110.0	Umbral=22.0
Episodio: 3 	Recompensa Media=23.56	Recompensa Max=73.0	Umbral=24.0
Episodio: 4 	Recompensa Media=21.22	Recompensa Max=92.0	Umbral=22.299999999999997
Episodio: 5 	Recompensa Media=22.87	Recompensa Max=74.0	Umbral=25.299999999999997
Episodio: 6 	Recompensa Media=22.59	Recompensa Max=82.0	Umbral=28.0
Episodio: 7 	Recompensa Media=24.16	Recompensa Max=113.0	Umbral=27.299999999999997
Episodio: 8 	Recompensa Media=23.9	Recompensa Max=80.0	Umbral=27.299999999999997
Episodio: 9 	Recompensa Media=20.5	Recompensa Max=56.0	Umbral=22.299999999999997
Episodio: 10 	Recompensa Media=25.63	Recompensa Max=86.0	Umbral=30.0
Episodio: 11 	Recompensa Media=20.97	Recompensa Max=88.0	Umbral=25.0
Episodio: 12 	Recompensa Media=22.07	Recompensa Max=57.0	Umbral=24.299999999999997
Episodio: 13 	R